In [1]:
import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

not
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [2]:
import pandas as pd

# transformers
import pytorch_transformers
from pytorch_transformers import *

# read file
from xml.dom import minidom
import re

#import rank model
from sklearn.ensemble import RandomForestRegressor

# custom extractor
import pickle
from my_functions import extractorRoberta
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_extractor = extractorRoberta(my_device = device, model_path = '/notebook/cqas/external_pretrained_models/')
print ("loaded extractors")

from pyterrier import text
from pyterrier.text import scorer

import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

def create_featured_dataset(some_df):
    from pyterrier import text
    from pyterrier.text import scorer
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
    rtr_bm = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    result = pd.merge(result, some_df, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"], result['baseline_scores'], result["is_retrieved"], result["ap_score"], result["objs_score"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result

def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)


def count_score1(text, nlu_tuple):
    (obj1, obj2, pred, asp) = nlu_tuple
    r = 1.0
    if (len(obj1) != 0 and len(obj2) != 0):
        if (len(pred) != 0):
            pred = re.sub('[!#?,.:";]', '', pred[0])
            if (obj1 in text and obj2 in text and pred in text):
                r += 1.0
        if (len(asp) != 0):
            asp = re.sub('[!#?,.:";]', '', asp[0])
            if (obj1 in text and obj2 in text and asp in text):
                r += 1.0
        elif (obj1 in text and obj2 in text):
            r = 1.5
        elif (obj1 in text or obj2 in text):
            r = 1.2
    else:
        if (obj1) in text or (obj2) in text:
            r = 1.2
    return r

def count_score_nlu(nlu_tuple):
    if (len(nlu_tuple[0]) == 0):
        return 0.0
    else: return 1.0


def count_score(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and len(obj2) != 0):
        if (obj1 in text):
            r += 1.0
        if (obj2 in text):
            r += 1.0
        for asp in asps:
            if asp in text:
                r += 1.5
        for pred in preds:
            if pred in text:
                r += 1.0
    else:
        if ((obj1) in text and len(obj1)!= 0) or (obj2 in text and len(obj2) != 0):
            r = 1.0
    return r

def count_score_obj(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and obj1 in text):
        r += 1.0
    if (len(obj2) != 0 and obj2 in text):
        r += 1.0
    return r

def count_score_asp_pred(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    o1 = (len(obj1) != 0 and obj1 in text)
    o2 = (len(obj2) != 0 and obj2 in text)
    if (o1 or o2):
        for asp in asps:
            if asp in text:
                r += 0.5
        for pred in preds:
            if pred in text:
                r += 0.5
    return r


def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

def make_a_search_request(query):
    # return json
    # json will be processed further
    params = {
            "apikey": "0833a307-97d3-462a-99d9-27db400c70da",
            "query": query,
            "index": ["cw12"],
            "size": 10,
            "pretty": True
        }
    response = requests.get(url = "http://www.chatnoir.eu/api/v1/_search", params = params)
    return response

def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


/notebook/cqas/external_pretrained_models/roberta.hdf5
/notebook/cqas/external_pretrained_models/vocab_dir
encoder loaded
indexer loaded
cuda device  cuda:0
model path  /notebook/cqas/external_pretrained_models/roberta.hdf5
model loaded
reader loaded
loaded extractors


In [3]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)
import requests
from xml.dom import minidom
import sys
import argparse

#numpy
import numpy as np

Read retireved documents and qrels:

In [4]:
topics_2020 = read_xml('topics-task-2.xml')
        
topics_2021 = read_xml('topics-task-2-only-titles-2021.xml')

1
[<DOM Element: topics at 0x7fed801d3cd0>]
50
1
[<DOM Element: topics at 0x7fed516c27d0>]
50


In [5]:
import pickle


with open('/notebook/touche/list_of_un_answ.pkl', 'rb') as f:
    answers_2020 = pickle.load(f)
    
with open('/notebook/touche2021/touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 


Create qrels dict:

In [6]:
qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))

In [7]:
qrels_dict['46'][:3]

[('clueweb12-1715wb-31-22014', '0'),
 ('clueweb12-0106wb-15-00715', '0'),
 ('clueweb12-1313wb-74-02250', '0')]

In [8]:
from allennlp.common.checks import ConfigurationError
from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.dataset_readers.dataset_utils import to_bioul
from allennlp.data.fields import TextField, SequenceLabelField, Field, MetadataField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token

from allennlp.modules.token_embedders import PretrainedTransformerMismatchedEmbedder
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.seq2seq_encoders import PassThroughEncoder



from allennlp.data.token_indexers import PretrainedTransformerMismatchedIndexer
from allennlp.data.vocabulary import Vocabulary
from allennlp.predictors import SentenceTaggerPredictor


In [9]:
import pickle
import pandas as pd

topics_2020 = read_xml('topics-task-2.xml')

topics_2021 = read_xml('topics-task-2-only-titles-2021.xml')
        
with open('list_of_un_answ.pcl', 'rb') as f:
    answers_2020 = pickle.load(f)
            
with open('list_of_un_answ_2021.pcl', 'rb') as f:
    answers_2021 = pickle.load(f)
    
with open('touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 

qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))
            
info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_train = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)
qrels_df_train = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

1
[<DOM Element: topics at 0x7fed516d4870>]
50
1
[<DOM Element: topics at 0x7fed484852d0>]
50


In [38]:
def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    if True:
        print ("sort")
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr

def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))
    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    return add_ranks1(test_DF)

In [31]:
info_df.head()

,qid,query,docno,text,baseline_scores,is_retrieved,ap_score,objs_score
0,51,what is better at reducing fever in children i...,clueweb12-0001wb-15-05765,"additionally, these medications can become tox...",1984.9130,1.0,0.0,0.0
1,51,what is better at reducing fever in children i...,clueweb12-0904wb-53-21907,but is your child&#x27;s fever really a cause ...,1700.4545,1.0,0.0,0.0
2,51,what is better at reducing fever in children i...,clueweb12-1314wb-72-19073,"in children, a fever that is equal to or great...",1472.7758,1.0,0.0,0.0
3,51,what is better at reducing fever in children i...,clueweb12-0311wb-02-29958,it is effective in reducing fever and is also ...,1469.3533,1.0,0.0,0.0
4,51,what is better at reducing fever in children i...,clueweb12-0003wb-21-31533,ibuprofen (nurofen and others) is another feve...,1104.5153,1.0,1.0,1.0


In [ ]:
info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_train = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)
qrels_df_train = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

for elem in topics_2021:
        qid, query = elem[0], elem[1].strip('\n')
        query = re.sub(r'[^\w\s]','',query)
        query = cleanhtml(query)
        my_extractor.from_string(query)
        structures = my_extractor.get_params()
        
        for ind, answer in enumerate(answers_2021[qid]):
            docno = answer[1]
            score = answer[0]
            text = answer[3]

            #nlu_score = count_score(text, structures)
            objs_score = count_score_obj(text, structures)
            ap_score = count_score_asp_pred(text, structures)
            is_retrieved = count_score_nlu(structures)
            df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
            info_df = info_df.append(df_row, ignore_index= True)
            
    #create train df and crels
    
for elem in topics_2020:
    qid, query = elem[0], elem[1].strip('\n')
    query = re.sub(r'[^\w\s]','',query)
    query = cleanhtml(query)
    my_extractor.from_string(query)
    structures = my_extractor.get_params()

    for ind, answer in enumerate(answers_2020[qid]):
        docno = answer[1]
        score = answer[0]
        text = answer[3]

        nlu_score = count_score(text, structures)
        objs_score = count_score_obj(text, structures)
        ap_score = count_score_asp_pred(text, structures)
        is_retrieved = count_score_nlu(structures)
        df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
        info_df_train = info_df_train.append(df_row, ignore_index= True)
                
    for qrel in qrels_dict[qid]:
        docno, label = qrel
        df_row = {"qid":qid, "docno":docno, "label":label}
        qrels_df_train = qrels_df_train.append(df_row, ignore_index= True)
           

In [32]:
print ("info_df head (2021) ", len(info_df), info_df.head())
print ("info_df_test head (2020) ", len(info_df_train), info_df_train.head())
print ("qrels_df_train ", len(qrels_df_train), qrels_df_train.head())
    
    
test_ds = create_featured_dataset(info_df)
result = create_featured_dataset(info_df_train)

info_df head (2021)  4912   qid                                              query  \
0  51  what is better at reducing fever in children i...   
1  51  what is better at reducing fever in children i...   
2  51  what is better at reducing fever in children i...   
3  51  what is better at reducing fever in children i...   
4  51  what is better at reducing fever in children i...   

                       docno  \
0  clueweb12-0001wb-15-05765   
1  clueweb12-0904wb-53-21907   
2  clueweb12-1314wb-72-19073   
3  clueweb12-0311wb-02-29958   
4  clueweb12-0003wb-21-31533   

                                                text  baseline_scores  \
0  additionally, these medications can become tox...        1984.9130   
1  but is your child&#x27;s fever really a cause ...        1700.4545   
2  in children, a fever that is equal to or great...        1472.7758   
3  it is effective in reducing fever and is also ...        1469.3533   
4  ibuprofen (nurofen and others) is another feve...   

In [19]:
result.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,features
0,1,clueweb12-0200wb-79-18105,12.977148,9.0,25.306646,23.477962,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.9827,0.0,0.0,0.0,"[12.977147771219041, 9.0, 25.306646413477576, ..."
1,1,clueweb12-1214wb-88-29751,12.706617,10.0,24.926572,23.321558,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.7341,0.0,0.0,0.0,"[12.706617142356468, 10.0, 24.926572135926463,..."
2,1,clueweb12-0706wb-92-20025,11.308204,6.0,23.195164,20.817880,what is the difference between sex and love,the prevailing attitude in our society is: let...,1402.0853,0.0,0.0,0.0,"[11.308203688644971, 6.0, 23.19516356116842, 2..."
3,1,clueweb12-1400tw-53-02910,8.896310,4.0,19.173446,17.210010,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.1604,0.0,0.0,0.0,"[8.896310063188475, 4.0, 19.173445870962855, 1..."
4,1,clueweb12-0406wb-58-08558,8.647981,5.0,17.805220,16.049722,what is the difference between sex and love,"sex, love and relationship are such a complex ...",1358.9696,0.0,0.0,0.0,"[8.64798145064058, 5.0, 17.80521998357908, 16...."


In [15]:
#print (info_df.head())
#test_ds = create_featured_dataset(info_df)
#result = create_featured_dataset(info_df_train)
#print ("\n\n test_ds")
#print (test_ds.head())    
#rf = RandomForestRegressor(n_estimators=20)
#rf_pipe = pt.ltr.apply_learned_model(rf)
#rf_pipe.fit(result, qrels_df)
#answs = transform(rf_pipe, test_ds)

In [21]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

rf_lgbm = pt.ltr.apply_learned_model(lmart_l)

In [42]:
answs.head(3)

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,features,score,rank
0,51,clueweb12-0504wb-21-02022,16.377481,7.0,34.188286,31.002816,what is better at reducing fever in children i...,"at home, fevers are more of a nuisance than a ...",1301.04480,1.0,1.0,2.0,"[16.377480718557777, 7.0, 34.18828622963671, 3...",0.154011,3
1,51,clueweb12-0604wb-54-09136,15.663725,7.0,33.405079,32.708491,what is better at reducing fever in children i...,see: acetaminophen overdose nsaids include asp...,710.59265,1.0,1.0,2.0,"[15.663725156371, 7.0, 33.40507866490724, 32.7...",-0.416655,21
2,51,clueweb12-0505wb-80-31263,13.260739,6.0,27.104483,24.583809,what is better at reducing fever in children i...,"by 6 hours, children treated with ibuprofen 5m...",449.02432,1.0,0.5,1.0,"[13.260739414244014, 6.0, 27.104483452764427, ...",-0.416655,22


In [40]:
test_ds.head(3)

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,features
0,51,clueweb12-0504wb-21-02022,16.377481,7.0,34.188286,31.002816,what is better at reducing fever in children i...,"at home, fevers are more of a nuisance than a ...",1301.04480,1.0,1.0,2.0,"[16.377480718557777, 7.0, 34.18828622963671, 3..."
1,51,clueweb12-0604wb-54-09136,15.663725,7.0,33.405079,32.708491,what is better at reducing fever in children i...,see: acetaminophen overdose nsaids include asp...,710.59265,1.0,1.0,2.0,"[15.663725156371, 7.0, 33.40507866490724, 32.7..."
2,51,clueweb12-0505wb-80-31263,13.260739,6.0,27.104483,24.583809,what is better at reducing fever in children i...,"by 6 hours, children treated with ibuprofen 5m...",449.02432,1.0,0.5,1.0,"[13.260739414244014, 6.0, 27.104483452764427, ..."


In [41]:

rf_lgbm.fit(result, qrels_df_train, topics_and_results_Valid = result, qrelsValid = qrels_df_train)
answs = transform(rf_lgbm, test_ds)

[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000362 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1052
[LightGBM] [Info] Number of data points in the train set: 4342, number of used features: 8
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [69]:
def write_qrels(output_dir, name, rtr):
    print ("rtr in write qrels ", rtr.head())
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    scores = rtr['score']
    ranks = rtr['rank']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, scores, tags))
    print ("common_list ", common_list[:3])
    with open(output_dir + name +'.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.txt')


In [68]:
answs.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,features,score,rank
0,51,clueweb12-0504wb-21-02022,16.377481,7.0,34.188286,31.002816,what is better at reducing fever in children i...,"at home, fevers are more of a nuisance than a ...",1301.04480,1.0,1.0,2.0,"[16.377480718557777, 7.0, 34.18828622963671, 3...",0.0,1
1,51,clueweb12-0604wb-54-09136,15.663725,7.0,33.405079,32.708491,what is better at reducing fever in children i...,see: acetaminophen overdose nsaids include asp...,710.59265,1.0,1.0,2.0,"[15.663725156371, 7.0, 33.40507866490724, 32.7...",0.0,2
2,51,clueweb12-0505wb-80-31263,13.260739,6.0,27.104483,24.583809,what is better at reducing fever in children i...,"by 6 hours, children treated with ibuprofen 5m...",449.02432,1.0,0.5,1.0,"[13.260739414244014, 6.0, 27.104483452764427, ...",0.0,3
3,51,clueweb12-1908wb-81-21824,12.717186,6.0,26.531794,23.993080,what is better at reducing fever in children i...,it works by reducing substances in the body th...,570.00920,1.0,1.0,1.0,"[12.717186432367184, 6.0, 26.531794229170387, ...",0.0,4
4,51,clueweb12-0410wb-39-29576,12.717186,6.0,26.531794,23.993080,what is better at reducing fever in children i...,it works by reducing substances in the body th...,558.62994,1.0,1.0,1.0,"[12.717186432367184, 6.0, 26.531794229170387, ...",0.0,5


In [70]:
write_qrels('/notebook/touche2021/output/', "lgbm_lmart", answs)

rtr in write qrels    qid                      docno  score_pl2  score_tf   score_bm  score_dfic  \
0  51  clueweb12-0504wb-21-02022  16.377481       7.0  34.188286   31.002816   
1  51  clueweb12-0604wb-54-09136  15.663725       7.0  33.405079   32.708491   
2  51  clueweb12-0505wb-80-31263  13.260739       6.0  27.104483   24.583809   
3  51  clueweb12-1908wb-81-21824  12.717186       6.0  26.531794   23.993080   
4  51  clueweb12-0410wb-39-29576  12.717186       6.0  26.531794   23.993080   

                                               query  \
0  what is better at reducing fever in children i...   
1  what is better at reducing fever in children i...   
2  what is better at reducing fever in children i...   
3  what is better at reducing fever in children i...   
4  what is better at reducing fever in children i...   

                                                text  baseline_scores  \
0  at home, fevers are more of a nuisance than a ...       1301.04480   
1  see: acetamin

In [148]:
rtr = qrels_df_test
qids = rtr['qid']
Q0s = [0 for elem in qids]
docs = rtr['docno']
ranks = rtr['label']

common_list = list(zip(qids, Q0s, docs, ranks))
with open("standart_test" +'.qrels', 'w') as fp:
    fp.write('\n'.join('%s %s %s %s' % x for x in common_list))
print ("written " + 'standart' +'_test.qrels')

written standart_test.qrels


In [76]:
qrels_df_test[:3]

,qid,docno,label
0,40,clueweb12-1608wb-39-15329,0
1,40,clueweb12-1700tw-30-03487,0
2,40,clueweb12-0811wb-62-19220,0


In [77]:
#write_qrels("standart", qrels_df_test)

In [78]:
cord19.get_topics(variant='train').head()

NameError: name 'cord19' is not defined

## LightGMB

In [52]:
cord19 = pt.datasets.get_dataset('antique')

In [55]:
topics = cord19.get_topics(variant='train')
qrels = cord19.get_qrels(variant='train')

In [57]:
import os
pt_index_path = './terrier_cord19_blocks'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path, blocks=True)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

In [58]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf = pt.BatchRetrieve(index, wmodel="Tf")
pl = pt.BatchRetrieve(index, wmodel="PL2")
dfic = pt.BatchRetrieve(index, wmodel="DFIC")

In [64]:
from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=15, random_state=42)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=5, random_state=42)


In [65]:
ltr_feats1 = (bm25 ** tf ** pl ** dfic)

# for reference, lets record the feature names here too
fnames=["BM25", "TF", 'pl', 'dfic']

In [219]:
import xgboost as xgb
# this configures XGBoost as LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
      learning_rate=0.1,
      gamma=1.0,
      min_child_weight=0.1,
      max_depth=6,
      verbose=2,
      random_state=42)

rf_lgbm = pt.ltr.apply_learned_model(lmart_x, form="ltr")
rf_lgbm.fit(result, qrels_df, topics_and_results_Valid = test_ds, qrelsValid = qrels_df_test)
answs = transform(rf_lgbm, test_ds)

0
1


Feature importance according LightGBM and XGboost

* f0: result["score_pl2"], 5, 1.76

* f1: result["score_tf"], 0, 1.19
* f2: result["score_bm"], 3, 1.51
* f3: result["score_dfic"], 3, 2.3
* f4: result['baseline_scores'], 19, 23.82
* f5: result["is_retrieved"], 0, 0
* f6: result["ap_score"], 3,  1.66
* f7: result["objs_score"]0, 1.51

In [29]:
rf_lgbm.learner.get_booster().get_score(importance_type="gain")

AttributeError: 'LGBMRanker' object has no attribute 'get_booster'

In [30]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=12,
    objective="lambdarank",
    metric="ndcg",
    learning_rate= .1,
    importance_type="gain",
    num_iterations=20,
    early_stopping_rounds=5
)

rf_lgbm = pt.ltr.apply_learned_model(lmart_l, form="ltr")
rf_lgbm.fit(result, qrels_df, topics_and_results_Valid = result, qrelsValid = qrels_df)
answs = transform(rf_lgbm, test_ds)



[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000427 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1052
[LightGBM] [Info] Number of data points in the train set: 4342, number of used features: 8
[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [28]:
rf_lgbm.learner.booster_.feature_importance()

array([0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [218]:
write_qrels("RF", answs)

written RF.qrels


## Custom transformers' score

In [229]:
from transformers_custom import make_scores_transformers, run_baseline

In [ ]:
for head_ in range(11):
    run_baseline(head = head_)

## Bert

In [6]:
import onir_pt

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
!pip3 install onir_pt

ERROR: Could not find a version that satisfies the requirement onir_pt (from versions: none)
ERROR: No matching distribution found for onir_pt
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
#!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
#!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

In [8]:
vbert = onir_pt.reranker('vanilla_transformer', 'bert', text_field='text', vocab_config={'train': True})